In [3]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import pandas as pd
import whisper
import sys
from moviepy.editor import *

In [4]:
name_person = "Hariom ss"

In [5]:
# tiny, base, small, medium, large: pick one model among these
model = whisper.load_model("small")

video_file_name = name_person + '.mp4'
audio_file_name = name_person + '_audio.mp3'

read_video_file = True
write_audio_file = True

if read_video_file:
    video = VideoFileClip(video_file_name)
    audio = video.audio
    if write_audio_file:
        audio.write_audiofile(audio_file_name)

MoviePy - Writing audio in Hariom ss_audio.mp3


MoviePy - Done.


In [ ]:
output = model.transcribe(name_person + '_audio.mp3', language='en')
transcription_df = pd.json_normalize(output['segments'])

transcript = output['text']

In [7]:
save_complete_text = True
if save_complete_text:
    with open(name_person + '_transcript.txt', 'w') as f:
        f.write(transcript)

# groupby seek can be used to know the stops and everything.
# for this test example we are getting 10 gaps in end_time.shift() - start_time()
# but 16 breaks using seek, which seem more appropriate
# transcription_df.drop(['id', 'tokens', 'temperature'], axis=1, inplace=True)
# transcription_df.to_csv('transcription_dataframe.csv', index=False)
# transcription_df.head()

In [12]:
with open('brijesh_audio2.txt', 'w') as f:
    f.write(transcript)

In [ ]:
# if you want to read the file only, then run this else skip this
# with open('test_transcript.txt', 'r') as f:
#     transcript = f.read()

In [ ]:
# transcript

In [5]:
## currently not working, dont know the reason
# whisper lanugage translation is not working well

# output_hindi = model.transcribe('test_audio.mp3', language="hi")

# output_hindi

KeyboardInterrupt: 

In [ ]:
# T5-base has only few languages available so we are using another pre-trained model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import stanza

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

In [ ]:
def translator(text):
    # function to translate english text to hindi
    input_ids = tokenizer.encode(text, return_tensors="pt", padding=True)
    outputs = model.generate(input_ids)
    decoded_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  
    return decoded_text


In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize')
doc = nlp(transcript)
sentences = []
for ele in doc.sentences:
#     print(ele)
    sentences.append(ele.text)


In [ ]:
hi_sentences = []
for text in sentences:
    print("English Text: ", text)
    hi = translator(text)
    hi_sentences.append(hi)
    print("Hindi Translation: ", hi)
    print("*"*50,"\n") 

issues: 
- too literal (e.g.: I hit escape.)
- too pure hindi
- some words need not be translated, but we are doing everyword in this. Like notebook -> टिप्पणीबुक
- unable to recognize the names: Jupyter, Fudge, etc

bad examples:

English Text:  You can also press shift and enter to execute the cell.
Hindi Translation:  आप कक्ष को चलाने के लिए भी शिफ्ट प्रेस और प्रवेश कर सकते हैं.

English Text:  This asterisk means that it's running.
Hindi Translation:  इसका मतलब है कि यह चल रहा है.

English Text:  I'll put in my name.
Hindi Translation:  मैं अपने नाम में डाल देंगे.

English Text:  I hit escape.
Hindi Translation:  मैं भागने मारा.

### Some other NMT model is required/ finetuning might also work. for that some data might be required. 


### TTS

In [ ]:
import os
from gtts import gTTS

In [ ]:
# converting text to wav
text = '. '.join(hi_sentences[:5])

# text='मैं अपना नाम डाल देंगे.'

language = 'hi' #hindi
speech = gTTS(text = text, lang = language, slow = False)
speech.save('medium_hindi.wav')


issues:
- Too robotic
- no control over accent, voice, male/female, fast/slow, etc